In [1]:
import pymysql
import pandas as pd
import numpy as np

In [2]:
from env import host, user, password

In [75]:
#Create a function named get_db_url. It should accept a username, 
#hostname, password, and database name and return a url connection 
#string formatted like in the example at the start of this lesson.

def get_db_url(db_name):
    from env import user, host, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [133]:
#- Use your function to obtain a connection to the employees database.
url = get_db_url("employees")

In [5]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [6]:
#- Once you have successfully run a query:

# a. Intentionally make a typo in the database url. What kind of error message do you see?

# b. Intentionally make an error in your SQL query. What does the error message look like?
#pd.read_sql('SELECT * FROM employes LIMIT 5 OFFSET 50', url)

ProgrammingError: (pymysql.err.ProgrammingError) (1146, "Table 'employees.employes' doesn't exist")
[SQL: SELECT * FROM employes LIMIT 5 OFFSET 50]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [9]:
# - Read the employees and titles tables into two separate DataFrames.
emp_df = pd.read_sql('SELECT * FROM employees', url)
titles_df = pd.read_sql('SELECT * FROM titles', url)

In [10]:
# - How many rows and columns do you have in each DataFrame? Is that what you expected?
emp_df.shape

(300024, 6)

In [11]:
titles_df.shape

(443308, 4)

In [27]:
# - Display the summary statistics for each DataFrame.
emp_df.describe()

,emp_no
count,300024.000000
mean,253321.763392
std,161828.235540
min,10001.000000
25%,85006.750000
50%,249987.500000
75%,424993.250000
max,499999.000000


In [28]:
titles_df.describe

<bound method NDFrame.describe of         emp_no            title   from_date     to_date
0        10001  Senior Engineer  1986-06-26  9999-01-01
1        10002            Staff  1996-08-03  9999-01-01
2        10003  Senior Engineer  1995-12-03  9999-01-01
3        10004         Engineer  1986-12-01  1995-12-01
4        10004  Senior Engineer  1995-12-01  9999-01-01
...        ...              ...         ...         ...
443303  499997         Engineer  1987-08-30  1992-08-29
443304  499997  Senior Engineer  1992-08-29  9999-01-01
443305  499998     Senior Staff  1998-12-27  9999-01-01
443306  499998            Staff  1993-12-27  1998-12-27
443307  499999         Engineer  1997-11-30  9999-01-01

[443308 rows x 4 columns]>

In [31]:
# - How many unique titles are in the titles DataFrame?
titles_df.title.unique()

array(['Senior Engineer', 'Staff', 'Engineer', 'Senior Staff',
       'Assistant Engineer', 'Technique Leader', 'Manager'], dtype=object)

In [37]:
# - What is the oldest date in the to_date column?
titles_df.to_date.sort_values().head()

16064     1985-03-01
406358    1985-03-11
346216    1985-03-20
35527     1985-03-23
277603    1985-04-17
Name: to_date, dtype: object

In [76]:
# - What is the most recent date in the to_date column?
titles_df[titles_df.to_date.astype('str') != '9999-01-01'].sort_values(by='to_date', ascending=False)

,emp_no,title,from_date,to_date
154568,204324,Technique Leader,1985-02-10,2002-08-01
119478,90666,Engineer,1997-08-01,2002-08-01
387242,461960,Staff,1997-08-01,2002-08-01
319569,416185,Staff,1999-10-28,2002-08-01
67905,55876,Staff,1995-08-01,2002-08-01
...,...,...,...,...
277603,287732,Staff,1985-04-14,1985-04-17
35527,34059,Senior Engineer,1985-02-10,1985-03-23
346216,434232,Staff,1985-02-26,1985-03-20
406358,474918,Engineer,1985-02-10,1985-03-11


In [77]:
# Exercises II
# Copy the users and roles DataFrames from the examples above.
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [78]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [80]:
# 2. What is the result of using a right join on the DataFrames?

users.merge(roles, left_on='role_id', right_on='id', how='right', indicator=True)
# Keeping 'indicator' in there for transparency

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1,admin,both
1,2.0,joe,2.0,2,author,both
2,3.0,sally,3.0,3,reviewer,both
3,4.0,adam,3.0,3,reviewer,both
4,NaN,NaN,NaN,4,commenter,right_only


In [83]:
# 3. What is the result of using an outer join on the DataFrames?

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

,id_x,name_x,role_id,id_y,name_y,_merge
0,1.0,bob,1.0,1.0,admin,both
1,2.0,joe,2.0,2.0,author,both
2,3.0,sally,3.0,3.0,reviewer,both
3,4.0,adam,3.0,3.0,reviewer,both
4,5.0,jane,NaN,NaN,NaN,left_only
5,6.0,mike,NaN,NaN,NaN,left_only
6,NaN,NaN,NaN,4.0,commenter,right_only


In [88]:
# 4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

# users.drop(columns='role_id').merge(roles.drop(columns='id'), left_on='role_id', right_on='id', how='outer')

# Key Error

KeyError: 'id'

In [89]:
# 5. Load the mpg dataset from PyDataset.

from pydataset import data
mpg = data('mpg')

In [194]:
# 6. Output and read the documentation for the mpg dataset.

data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




In [161]:
# 7. How many rows and columns are in the dataset?

mpg.shape

(234, 14)

In [213]:
# 8. Check out your column names and perform any cleanup you may want on them.

mpg.rename(columns={'manufacturer' : 'Manufacturer',
                   'model' : 'Model',
                   'displ' : 'Displacement (L)',
                   'year' : 'Year',
                    'cyl' : 'Cylinders',
                   'trans' : 'Transmission',
                   'drv' : 'Drive',
                   'cty' : 'City',
                   'hwy' : 'Highway',
                   'fl' : 'Fl',
                   'class' : 'Body Type'})

,Manufacturer,Model,Displacement (L),Year,Cylinders,Transmission,Drive,City,Highway,Fl,Body Type,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


In [160]:
# 9. Display the summary statistics for the dataset.

mpg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   manufacturer        234 non-null    object 
 1   model               234 non-null    object 
 2   displ               234 non-null    float64
 3   year                234 non-null    int64  
 4   cyl                 234 non-null    int64  
 5   trans               234 non-null    object 
 6   drv                 234 non-null    object 
 7   cty                 234 non-null    int64  
 8   hwy                 234 non-null    int64  
 9   fl                  234 non-null    object 
 10  class               234 non-null    object 
 11  mileage_difference  234 non-null    int64  
 12  average_mileage     234 non-null    float64
 13  is_automatic        234 non-null    bool   
dtypes: bool(1), float64(2), int64(5), object(6)
memory usage: 25.8+ KB


In [212]:
mpg.describe().round(2)

,displ,year,cyl,cty,hwy,mileage_difference,average_mileage
count,234.00,234.00,234.00,234.00,234.00,234.00,234.00
mean,3.47,2003.50,5.89,16.86,23.44,6.58,20.15
std,1.29,4.51,1.61,4.26,5.95,2.26,5.05
min,1.60,1999.00,4.00,9.00,12.00,2.00,10.50
25%,2.40,1999.00,4.00,14.00,18.00,5.00,15.50
50%,3.30,2003.50,6.00,17.00,24.00,7.00,20.50
75%,4.60,2008.00,8.00,19.00,27.00,8.00,23.50
max,7.00,2008.00,8.00,35.00,44.00,12.00,39.50


In [103]:
# 10. How many different manufacturers are there?

mpg.manufacturer.nunique()

15

In [104]:
# 11. How many different models are there?

mpg.model.nunique()

38

In [106]:
# 12. Create a column named mileage_difference like you did in 
# the DataFrames exercises; this column should contain the difference 
# between highway and city mileage for each car.

mpg = mpg.assign(mileage_difference=(mpg.hwy - mpg.cty))

In [107]:
# 13. Create a column named average_mileage like you did in the DataFrames 
# exercises; this is the mean of the city and highway mileage.

mpg = mpg.assign(average_mileage=((mpg.hwy + mpg.cty)/2))

In [164]:
# 14. Create a new column on the mpg dataset named is_automatic that holds 
# boolean values denoting whether the car has an automatic transmission.

mpg = mpg.assign(is_automatic = mpg.trans.str.startswith('a'))

In [168]:
# 15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

mpg.groupby('manufacturer').average_mileage.agg('mean').sort_values(ascending=False).head(1)

manufacturer
honda    28.5
Name: average_mileage, dtype: float64

In [123]:
# 16. Do automatic or manual cars have better miles per gallon?

mpg.groupby('is_automatic').average_mileage.mean() (#or .agg('mean'))

is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

In [200]:
# Exercises III
# 1. Use your get_db_url function to help you explore the data from the chipotle database.

url2 = get_db_url("chipotle")
chipotle = pd.read_sql('SELECT * FROM orders', url2)

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98
...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75


In [201]:
# 2. What is the total price for each order?
# Convert price to float

chipotle = chipotle.assign(item_total = chipotle.item_price.str.strip('$').astype('float'))

,id,order_id,quantity,item_name,choice_description,item_price,item_total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39,2.39
1,2,1,1,Izze,[Clementine],$3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98,16.98
...,...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75,11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",$11.75,11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",$11.25,11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",$8.75,8.75


In [202]:
chipotle.groupby('order_id').item_total.agg('sum') #or .sum()

order_id
1       11.56
2       16.98
3       12.67
4       21.00
5       13.70
        ...  
1830    23.00
1831    12.90
1832    13.20
1833    23.50
1834    28.75
Name: item_total, Length: 1834, dtype: float64

In [203]:
# 3. What are the most popular 3 items?

chipotle.groupby('item_name').sum('quantity').sort_values(by='quantity', ascending=False).head(3)

,id,order_id,quantity,item_total
item_name,,,,
Chicken Bowl,1780635,713926,761,7342.73
Chicken Burrito,1238770,497303,591,5575.82
Chips and Guacamole,1122252,449959,506,2201.04


In [204]:
# 4. Which item has produced the most revenue?

chipotle.groupby('item_name').item_total.agg('sum').sort_values(ascending=False).head(5)

item_name
Chicken Bowl           7342.73
Chicken Burrito        5575.82
Steak Burrito          3851.43
Steak Bowl             2260.19
Chips and Guacamole    2201.04
Name: item_total, dtype: float64

In [188]:
# 5. Join the employees and titles DataFrames together.

# emp_titles = pd.merge(emp_df, titles_df, left_on='emp_no', right_on='emp_no', how='inner')
# Messed around with both to see how it handled defaults.

emp_titles = pd.merge(emp_df, titles_df, how='inner')


,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...
443303,True,True,True,True,True,True,True,True,True
443304,True,True,True,True,True,True,True,True,True
443305,True,True,True,True,True,True,True,True,True
443306,True,True,True,True,True,True,True,True,True


In [184]:
# 6. For each title, find the hire date of the employee that was hired most recently with that title.

emp_titles.groupby('title').hire_date.agg('max')

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

In [211]:
# 7. Write the code necessary to create a cross tabulation of the number of titles 
# by department. (Hint: this will involve a combination of SQL code to pull the necessary 
# data and python/pandas code to perform the manipulations.)

query = '''

SELECT emp_no, title, dept_name

FROM employees
JOIN titles USING(emp_no)
JOIN dept_emp USING(emp_no)
JOIN departments USING(dept_no)

where titles.to_date = "9999-01-01"

'''

temp = pd.read_sql(query, url)

pd.crosstab(temp.title, temp.dept_name)


dept_name,Customer Service,Development,Finance,Human Resources,Marketing,Production,Quality Management,Research,Sales
title,,,,,,,,,
Assistant Engineer,68,1833,0,0,0,1542,433,77,0
Engineer,627,15677,0,0,0,13325,3744,830,0
Manager,1,1,1,1,1,1,1,1,1
Senior Engineer,1790,43364,0,0,0,37156,10390,2250,0
Senior Staff,12349,1085,10650,10843,12371,1123,0,10219,32032
Staff,3902,315,3199,3416,3880,349,0,3206,9967
Technique Leader,241,6117,0,0,0,5210,1422,321,0
